In [ ]:
from shapely.geometry import Polygon, LineString, Point

import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
site = pd.read_excel('sample-folder/XL-EID 4G Capacity Sector W26-W29 2023_edit.xlsb', header=5, sheet_name='Sector')
site = gpd.GeoDataFrame(site, geometry=gpd.points_from_xy(site['Longitude'], site['Latitude']), crs=4326)
site = site.to_crs(3857)

In [ ]:
# This algorithm will get multiple output of nearest site - Please note! This algorith requires a unique tower ID/Site ID.

combined = pd.DataFrame()

for x in np.unique(site['Tower ID Special']):
    table = site[site['Tower ID Special'].str.contains(x, regex=False) == False].reset_index(drop=True)
    
    point = {'geometry': site[site['Tower ID Special'].str.contains(x, regex=False) == True]['geometry'].reset_index(drop=True).to_list() * len(table.index), 'Tower_ID': [x] * len(table.index)}

    point = gpd.GeoDataFrame(point,crs=3857)
    # point = point.set_crs(3857)
    distance = table.distance(point)

    combine = pd.concat([pd.DataFrame(table), pd.DataFrame(point), pd.DataFrame({'distance': distance})], axis=1).drop(columns=['geometry'])
    combine = combine[combine['distance'] < 500]
    combined = pd.concat([combined, combine], ignore_index=True) 

In [ ]:
# This algorithm only get 1 nearest site - Please note! Sjoin_nearest will output multiple result  if they have same distance in second/third/others nearest point/polygon 

# combined = pd.DataFrame()

# for x in np.unique(site['Tower ID Special']):
    
#     combine = site[site['Tower ID Special'].str.contains(x) ==True].sjoin_nearest(site[site['Tower ID Special'].str.contains(x) == False ], how='left', max_distance=50000, distance_col='distance')

#     combine =  pd.DataFrame(combine)
    
#     combined = pd.concat([combined, combine], ignore_index=True) 


In [ ]:
combined.to_excel('result/test.xlsx', index=False)